# Final Evaluation 

In this notebook the final resut is going to be evaluated. The data is first loaded into the Notebokk and then loaded into the current database.
After that the pipeline in the scope of this thesis is applied and the results collected for final evaluation

#### Loading the data

In [33]:
import pandas
import numpy
import re
import time
import csv
import os

from Main.combined_pipeline import combined_pipeline
path="Data/itunes_amazon_raw_data/labeled_data.csv"

#Read the file
data=pandas.read_csv(path, skiprows=5, encoding='unicode_escape')
data


,Unnamed: 0,_id,ltable.Sno,rtable.Sno,ltable.Album_Name,ltable.Artist_Name,ltable.CopyRight,ltable.Released,ltable.Song_Name,ltable.Time,rtable.Album_Name,rtable.Artist_Name,rtable.CopyRight,rtable.Released,rtable.Song_Name,rtable.Time,label
0,916,916,111,53124,vhs,x ambassadors,2015 kidinakorner/interscope records,30-Jun-15,vhs outro (interlude),1:25,vhs [explicit],x ambassadors,(c) 2015 kidinakorner/interscope records,"June 30, 2015",vhs outro (interlude) [explicit],1:25,1
1,1053,1053,148,50767,title (deluxe),meghan trainor,"2014, 2015 epic records, a division of sony m...",9-Jan-15,credit,2:51,title (deluxe),meghan trainor,"2011 what a music ltd, licence exclusive parl...","January 9, 2015",credit,2:51,1
2,1290,1290,206,41214,slow down (remixes),selena gomez,"2013 hollywood records, inc.",20-Aug-13,slow down (smash mode remix),5:21,slow down remixes,selena gomez,"(c) 2013 hollywood records, inc.","August 20, 2013",slow down (smash mode remix),5:21,1
3,1424,1424,211,19812,slow down (reggae remixes) - single,selena gomez,"2013 hollywood records, inc.",20-Aug-13,slow down (sure shot rockers reggae dub remix),3:15,good for you (remixes),selena gomez,(c) 2015 interscope records,"September 4, 2015",good for you (yellow claw & cesqeaux remix) [f...,3:01,0
4,1706,1706,250,53111,vhs,x ambassadors,2015 kidinakorner/interscope records,30-Jun-15,vhs outro (interlude),1:25,vhs [explicit],x ambassadors,(c) 2015 kidinakorner/interscope records,"June 30, 2015",first show (interlude),0:11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,57960,57960,6867,13598,doggumentary,snoop dogg,"__æ 2011 capitol records, llc. all rights re...",29-Mar-11,peer pressure (feat. traci nelson),4:07,doggumentary [edited],snoop dogg,"(c) 2011 capitol records, llc","March 29, 2011",boom ((feat. t-pain) [edited]),3:50,0
535,57993,57993,6868,13597,doggumentary,snoop dogg,"__æ 2011 capitol records, llc. all rights re...",29-Mar-11,i don't need no b**** (feat. devin the dude & ...,3:59,doggumentary [edited],snoop dogg,"(c) 2011 capitol records, llc","March 29, 2011",platinum ((feat. r. kelly) [edited]),4:29,0
536,58289,58289,6877,13623,doggumentary,snoop dogg & gorillaz,"__æ 2011 capitol records, llc. all rights re...",29-Mar-11,sumthin like this night (feat. gorillaz),3:37,doggumentary [explicit],snoop dogg,"(c) 2011 capitol records, llc","March 29, 2011",this weed iz mine (feat. wiz khalifa) [explicit],3:43,0
537,58403,58403,6882,13611,doggumentary,snoop dogg & latoiya williams,"_Ê 2011 capitol records, llc. all rights re...",29-Mar-11,cold game (feat. latoiya williams),3:49,doggumentary [explicit],snoop dogg,"(c) 2011 capitol records, llc","March 29, 2011",toyz n da hood (feat. bootsy collins) [explicit],2:40,0


In [34]:
class_vector=data['label']
left_table=data.iloc[:,4:10]
right_table=data.iloc[:,10:16]

In [35]:
#Import the necessary parts from the pipeline
from Evaluation.evaluation import evaluate_results
from Main.combined_pipeline import combined_pipeline
from Evaluation.test_evaluation import evaluate_results
from Utilities.database import query_database

In [36]:
#Function for populating the database
def create_and_populate_table(dataframe, table_name):
    try:
        # Clean table name (optional)
        table_name = table_name.replace(" ", "_")

        # Drop table if it exists
        delete_table_query = f"DROP TABLE IF EXISTS {table_name} CASCADE;"
        query_database(delete_table_query, printing=False)

        # Convert all data to string format
        dataframe = dataframe.astype(str)

        # Create table with a single column
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (Aggregate TEXT);"
        query_database(create_table_query, printing=False)
        print(f"Table '{table_name}' created successfully.")

        # Insert data into table
        values = []
        for _, row in dataframe.iterrows():
            row_values = "\n".join(row.astype(str))  # Concatenate row values with newline separator
            values.append(f"('{row_values.replace("'", "''")}')")  # Escape single quotes for SQL

        if values:
            insert_query = f"INSERT INTO {table_name} VALUES {', '.join(values)};"
            query_database(insert_query, printing=False)
            print(f"Table '{table_name}' populated successfully.")
        else:
            print("No data to insert.")

    except Exception as e:
        print(f"An error occurred: {e}")

In [37]:
#Function for the definition of the BLEU score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def tuple_bleu_score(candidate, reference, n_gram=1):
    """
    Compute BLEU score between two tuples treated as sentences.
    n_gram: how many grams to consider for BLEU (1, 2, 3, etc.)
    """
    weights = tuple((1.0 / n_gram for _ in range(n_gram)))  # e.g., (1.0,) for 1-gram, (0.5, 0.5) for 2-gram
    chencherry = SmoothingFunction()
    return sentence_bleu([reference], candidate, weights=weights, smoothing_function=chencherry.method1)

def average_best_bleu_score(answer, required_answer, n_gram=1):
    """
    For each tuple in 'answer', find the best BLEU match in 'required_answer'.
    Return average of these best scores.
    """
    if not answer or not required_answer:
        return 0.0  # No data to compare

    best_scores = []
    for candidate in answer:
        best_score = 0.0
        for reference in required_answer:
            score = tuple_bleu_score(candidate, reference, n_gram=n_gram)
            best_score = max(best_score, score)
        best_scores.append(best_score)

    final_score = sum(best_scores) / len(best_scores)
    return final_score

# Example data
answer = {('apple', 'banana'), ('carrot', 'date')}
required_answer = {('apple', 'date'), ('banana', 'carrot')}

In [38]:
#Define the index range
index_start = 0
index_end = 10

execution_mode = "local"
#execution_mode = "global"
#Running locally
if execution_mode == "local":
    create_and_populate_table(left_table.iloc[index_start:index_end, :], "left_table")
    create_and_populate_table(right_table.iloc[index_start:index_end, :], "right_table")
    ground_truth_left = data.iloc[index_start:index_end, ]
    ground_truth_left = ground_truth_left[ground_truth_left["label"] == 1]
    ground_truth_left = ground_truth_left.iloc[:, 4:10]
    

#4-10
else: 
    create_and_populate_table(left_table, "left_table")
    create_and_populate_table(right_table, "right_table")
    ground_truth_left = data
    ground_truth_left = ground_truth_left[ground_truth_left["label"] == 1]
    ground_truth_left = ground_truth_left.iloc[:, 4:10]



create_and_populate_table(ground_truth_left, "ground_truth_left")
ground_truth_answer=query_database("SELECT * FROM ground_truth_left;", printing=False)
ground_truth_answer

Table 'left_table' created successfully.
Table 'left_table' populated successfully.
Table 'right_table' created successfully.
Table 'right_table' populated successfully.
Table 'ground_truth_left' created successfully.
Table 'ground_truth_left' populated successfully.


[('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',),
 ('title (deluxe)\nmeghan trainor\n 2014, 2015 epic records, a division of sony music entertainment\n9-Jan-15\ncredit\n2:51',),
 ('slow down (remixes)\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (smash mode remix)\n5:21',),
 ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',),
 ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',),
 ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',)]

In [39]:
def calculate_write(two_step_values, threshold_series):
    for threshold in threshold_series:
        for two_step in two_step_values:

            # Execute and write the output to the file
            start = time.time()
            try:
                answer, metadata = combined_pipeline(query=None, initial_sql_query="SELECT * FROM right_table JOIN left_table ON left_table.aggregate = right_table.aggregate;", aux=True, threshold=threshold, two_step=two_step)
            except Exception as e:
                print(f"An error occurred: {e}")
                metadata = {'prompt_token_count': 0, 'candidates_token_count': 0, 'total_token_count': 0, 'total_calls': 0}
            end = time.time()

            # Problematic situation solution prompt
            solution_prompt = "SELECT * FROM left_table INNER JOIN left_tableaggregateright_tableaggregate_table ON left_table.aggregate = left_tableaggregateright_tableaggregate_table.word INNER JOIN right_table ON left_tableaggregateright_tableaggregate_table.synonym = right_table.aggregate;"

            # Try to get the answer
            required_answer = set()
            try:
                required_answer = query_database(solution_prompt, printing=False)
            except:
                required_answer = set()

            # correct_answer = required_answer == answer
            unique_result = list(set(tuple(sorted(set(t))) for t in required_answer))

            try:
                accuracy, precision, recall, f1_score = evaluate_results(ground_truth_answer, unique_result)
            except:
                accuracy, precision, recall, f1_score = 0, 0, 0, 0

            n_gram_range = [1, 2, 3, 4]
            bleu_scores = {}

            for n_gram in n_gram_range:
                score = average_best_bleu_score(ground_truth_answer, required_answer, n_gram=n_gram)
                bleu_scores[n_gram] = score  # Store in dictionary
                print(f"BLEU-{n_gram} score: {score:.4f}")  # Nicely formatted output

            # --- Write to CSV ---
            csv_row = {
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "f1_score": f1_score,
                "execution_time": end - start,
                "threshold": threshold,
                "two_step": two_step,
                "bleu_1": bleu_scores[1],
                "bleu_2": bleu_scores[2],
                "bleu_3": bleu_scores[3],
                "bleu_4": bleu_scores[4],
                "prompt_token_count": metadata['prompt_token_count'],
                "candidates_token_count": metadata['candidates_token_count'],
                "total_token_count": metadata['total_token_count'],
                "total_calls": metadata['total_calls']
            }

            csv_file = "results_output.csv"

            # Check if file exists to decide whether to write headers
            file_exists = os.path.isfile(csv_file)

            # Write data
            with open(csv_file, mode='a', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=csv_row.keys())
                if not file_exists:
                    writer.writeheader()  # Write headers if file doesn't exist
                writer.writerow(csv_row)

In [40]:
# Series of threshold
threshold_series = []
i = 0
while i < 1:
    i += 1 / 8
    threshold_series.append(i)

#threshold_series=[0.75, 0.875, 1.0]
# For testing purposes
two_step_values = [True, False]
#threshold_series = [0.9]

calculate_write(two_step_values, threshold_series)

two_step_values = [None]
threshold_series = [0.9]

calculate_write(two_step_values, threshold_series)

The final answer to the query is []
--------------------------------------
The 
SELECT * FROM right_table JOIN left_table ON left_table.aggregate = right_table.aggregate;
 has a JOIN clause.
[['SELECT aggregate FROM left_table;', "(2, <Comparison '=' at 0x7AD37C4ACB20>)", 'left_table.aggregate = right_table.aggregate', 'SELECT aggregate FROM right_table;']]
The final answer to the query is [('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',), ('title (deluxe)\nmeghan trainor\n 2014, 2015 epic records, a division of sony music entertainment\n9-Jan-15\ncredit\n2:51',), ('slow down (remixes)\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (smash mode remix)\n5:21',), ('slow down (reggae remixes) - single\nselena gomez\n 2013 hollywood records, inc.\n20-Aug-13\nslow down (sure shot rockers reggae dub remix)\n3:15',), ('vhs\nx ambassadors\n 2015 kidinakorner/interscope records\n30-Jun-15\nvhs outro (interlude)\n1:25',), (

KeyboardInterrupt: 